In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from urllib.parse import quote
import os

In [2]:
load_dotenv(dotenv_path="password.env")

host=os.getenv("host")
port=3306
user=os.getenv("user")
database=os.getenv("name")
password=os.getenv("password")
password_embedded = quote(password)
password_embedded = quote(password)  

conn = pymysql.connect(
    host=host,
    port=3306,
    user=user,
    database=database,
    password=password)

# Connecting to SQL

connection = create_engine(f"mysql+pymysql://{user}:{password_embedded}@{host}:{port}/{database}")

In [4]:
# Reading in CSVs

inventory = pd.read_csv('inventory.csv')
books = pd.read_csv('books.csv')
authors = pd.read_csv('authors.csv')
patrons = pd.read_csv('patrons.csv')
loans = pd.read_csv('loans.csv')
books['isbn'] = books['isbn'].astype(str)
authors['author_id'] = authors['author_id'].astype('int64')
books['author_id'] = books['author_id'].astype('int64')

In [5]:
# To SQL

inventory.to_sql(name="inventory", con=connection, if_exists='replace', index=False)
books.to_sql(name="books", con=connection, if_exists='replace', index=False)
authors.to_sql(name="authors", con=connection, if_exists='replace', index=False)
patrons.to_sql(name="patrons", con=connection, if_exists='replace', index=False)
loans.to_sql(name="loans", con=connection, if_exists='replace', index=False)

50

In [1]:
#Queries

query1 = "SELECT DISTINCT(Books.Title), Books.pub_year FROM Inventory JOIN Books ON Inventory.ISBN = Books.ISBN ORDER BY Books.pub_year LIMIT 5;"
query2 = "SELECT Books.Title AS Title, COUNT(Inventory.isbn) AS Count FROM Inventory LEFT JOIN Books ON Inventory.isbn = Books.isbn GROUP BY Books.Title;"
query3 = "SELECT Books.Title AS Title, COUNT(Inventory.ISBN) AS Count FROM Inventory LEFT JOIN Books ON Inventory.ISBN = Books.ISBN LEFT JOIN Loans ON Inventory.book_id = Loans.book_id WHERE Loans.return_date IS NULL GROUP BY Books.Title;"
query4 = "SELECT Books.Genre AS Genre, COUNT(Inventory.ISBN) AS Book_Count FROM Inventory LEFT JOIN Books ON Inventory.ISBN = Books.ISBN GROUP BY Books.Genre;"
query5 = "SELECT Books.Genre AS Genre, COUNT(DISTINCT(Books.ISBN)) AS Title_Count FROM Inventory LEFT JOIN Books ON Inventory.ISBN = Books.ISBN GROUP BY Books.Genre;"
query6 = "SELECT Authors.language, COUNT(*) AS number FROM Inventory JOIN Books ON Inventory.isbn = Books.isbn JOIN Authors ON Authors.author_id = Books.author_id GROUP BY Authors.language ORDER BY number;"
query7 = "SELECT `condition`, COUNT(*) FROM Inventory GROUP BY `condition`;"
query8 = "SELECT Books.title, MAX(CASE WHEN Inventory.Condition = 'As New' OR Inventory.Condition = 'good' THEN 1 ELSE 0 END) AS has_new_book FROM Books JOIN Inventory ON Books.ISBN = Inventory.ISBN GROUP BY Books.title;"
query9 = "SELECT CONCAT(Authors.first_name, ' ', Authors.last_name) AS full_name, COUNT(*) AS number_of_books FROM Inventory JOIN Books ON Inventory.isbn = Books.isbn JOIN authors ON books.author_id = Authors.author_id GROUP BY full_name;"
query10 = "SELECT Books.Title, COUNT(*) AS loan_count FROM Inventory JOIN Books ON Inventory.isbn = Books.isbn JOIN Loans ON Inventory.book_id = Loans.book_id GROUP BY Books.Title ORDER BY loan_count;"